# Working with text data

Here we will see how to prepare input text for training LLMs. This involves splitting the text into individual word and subword tokens, which can be encoded into vector representations for the LLM.

## 2.1 Understanding word embeddings

Deep neural network models, including LLMs, cannot process raw text directly. Therefore, we need a way to represent words as continous-valued vectors.

The concept of converting data into a vector format is often referred to as embedding.

Word embeddings can have varying dimensions, from one to thousands.

## 2.2 Tokenizing text

Let's see how we split input text into individual tokens, a required preprocessing step for creating embeddings for an LLM.

We start with a simple text and Python’s `re.split` function to split the text while keeping the delimiters:

In [ ]:
import re

text = "Hello, world. Is this-- a test?"
result = re.split(r'([,.:;?_!"()\']|--|\s)', text)    # capturing group (...) with three alternatives, single characters [...], -- and whitespace
result = [item.strip() for item in result if item.strip()]
print(result)

In [ ]:
import os
import requests

file_path = "the-verdict.txt"

if not os.path.exists(file_path):
    url = (
        "https://raw.githubusercontent.com/rasbt/"
        "LLMs-from-scratch/main/ch02/01_main-chapter-code/"
        "the-verdict.txt"
    )
    response = requests.get(url)
    response.raise_for_status()
    with open(file_path, "wb") as f:
        f.write(response.content)

In [ ]:
with open(file_path, "r", encoding='utf-8') as f:
    raw_text = f.read()
print(f'Total number of characters: {len(raw_text)}')
print(raw_text[:99])

Now let's apply our basic tokenizer to the main text:

In [ ]:
preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
preprocessed = [item.strip() for item in preprocessed if item.strip()]
print(f'Number of tokens in the text: {len(preprocessed)}')
print(f'Number of unique tokens in the text: {len(set(preprocessed))}')
print(f'First 30 tokens in the text:\n{preprocessed[:30]}')

## 2.3 Converting tokens into IDs

Next let's convert these tokens from a Python string to an integer representation to produce the token IDs.

To do this we need to build a vocabulary. This defines how we map each unique token to a unique integer.

In [ ]:
all_words = sorted(set(preprocessed))
vocab_size = len(all_words)
print(f'Vocabulary size: {vocab_size}')

In [ ]:
str_to_int = {token: i for i, token in enumerate(all_words)}    #
for i, item in enumerate(str_to_int.items()):
    print(f'{i}: {item}')
    if i >= 50:
        break

We need also a way to turn token IDs into text. For this we create an inverse version of the vocabulary that maps token IDs back to text tokens:

In [ ]:
int_to_str = {i: s for s, i in str_to_int.items()}
print(int_to_str[50])

We are now ready to implement a complete tokenizer class in Python with the following features:
- an `encode` method that splits text into tokens and carries out the string-to-integer mapping.
- a `decode` method that carries out the reverse integer-to-string mapping to convert token IDs back to text.

In [ ]:
class SimpleTokenizerV1:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i: s for s, i in vocab.items()}

    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids

    def decode(self, ids):
        text = [self.int_to_str[i] for i in ids]
        text = ' '.join(text)
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)    # remove extra spaces before punctuation.
        return text


We can now use the tokenizer to encode texts into integers:

In [ ]:
tokenizer = SimpleTokenizerV1(vocab=str_to_int)
text = """"It's the last he painted, you know,"Mrs. Gisburn said with pardonable pride."""
ids = tokenizer.encode(text)
print(ids)

Let's now try to turn these token IDs back into text using the `decode` method:

In [ ]:
print(tokenizer.decode(ids))

Based on this output we can see that the decode method worked successfully.

Let's now apply our tokenizer to text sample not contained in the training set:

In [ ]:
text = "Hello, do you like tea?"
print(tokenizer.encode(text))
print(tokenizer.decode(tokenizer.encode(text)))

We got `str_to_int['Hello']` key error. The problem is that the word "Hello" was not used in the original text. Hence, it is not contained in the vocabulary.

## 2.4 Adding special context tokens

We need to modify the tokenizer to handle unknown words. We also need to address the usage and addition of special context tokens that can enhance a models understanding of context or other relevant information of the text.

We will modify the vocabulary and tokenizer to support two new tokens `<|unk|>` and `<|endoftext|>`.

Let's start by extending our vocabularly with the new tokens `<|unk|>` and `<|endoftext|>`:

In [ ]:
all_tokens = sorted(set(preprocessed))
all_tokens.extend(['<|unk|>', '<|endoftext|>'])
str_to_int = {s: i for i, s in enumerate(all_tokens)}
print(f'Vocabulary size: {len(str_to_int)}')

In [ ]:
for s, i in list(str_to_int.items())[-5:]:
    print(f'{i}: {s}')

We confirm that the two new special tokens were successfully incorporated into the vocabulary.

We are now ready to implement a complete tokenizer class in Python with the following features:
- an `encode` method that splits text into tokens and carries out the string-to-integer mapping.
- unknown words that are not part of the vocabulary must be mapped to special token `<|unk|>`.
- independent text sources must be separated by special token `<|endoftext|>`.
- a `decode` method that carries out the reverse integer-to-string mapping to convert token IDs back to text.


In [ ]:
class SimpleTokenizerV2:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i: s for s, i in vocab.items()}

    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        preprocessed = [item.strip() if item in self.str_to_int else '<|unk|>' for item in preprocessed]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids

    def decode(self, ids):
        text = [self.int_to_str[i] for i in ids]
        text = ' '.join(text)
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)    # remove extra spaces before punctuation.
        return text


Let's now apply the tokenizer to a new text sample not contained in the training set:

In [ ]:
text = "Hello, do you like tea?"
tokenizer = SimpleTokenizerV2(vocab=str_to_int)
print(tokenizer.encode(text))
print(tokenizer.decode(tokenizer.encode(text)))

The first word Hello has successfully been mapped to token ID 1130 and back to `<|unk|>`.

Let's now try to combine two independent texts:

In [ ]:
text1 = "Hello, do you like tea?"
text2 = "In the sunlit terraces of the palace."

text = " <|endoftext|> ".join((text1, text2))

print(text)

In [ ]:
print(tokenizer.encode(text))
print(tokenizer.decode(tokenizer.encode(text)))

## 2.5 Byte pair encoding

The Byte Pair Encoder (BPE) was used to train LLMs such as GPT-2, GPT-3, and the original model used in ChatGPT.

Let's first look at an existing implementation from the tiktoken library:

In [ ]:
import tiktoken

tokenizer = tiktoken.get_encoding('gpt2')
text = "Hello, do you like tea? <|endoftext|> In the sunlit terraces of someunknownPlace."

In [ ]:
integers = tokenizer.encode(text, allowed_special={"<|endoftext|>"})
print(integers)
strings = tokenizer.decode(integers)
print(strings)

The encoding and decoding looks good.

Specifically, we see that the BPE tokenizer managed to encode and decode unknown words such as someunknowPlace correctly.

The algorithm underlying BPE breaks down words that aren't in its predefined vocabulary into smaller subword units or even individual characters, enabling it to handle out-of-vocabularly words.

Let's take a closer look:

In [ ]:
tokens = tokenizer.encode('someunknownPlace')
print(tokens)

In [ ]:
for token in tokens:
    print(token, tokenizer.decode([token]))

How does it do this? BPE build its vocabulary by iteratively merging frequent characters into subwords and frequent subwords into words.

In [ ]:
tokenizer.decode([11246])